<a href="https://colab.research.google.com/github/Felixaustine/titanic-survival-prediction/blob/main/Titanic_ML_Passanger_Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

titanic_path = kagglehub.competition_download('titanic')
yingfeng_titanic_train_path = kagglehub.notebook_output_download('yingfeng/titanic-train')
soserian_titanic_test_path = kagglehub.notebook_output_download('soserian/titanic-test')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df  = pd.read_csv("/kaggle/input/titanic/test.csv")

print(train_df.shape)
print(test_df.shape)


In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

print("Train Columns:", train_df.columns.tolist())
print("Test Columns:", test_df.columns.tolist())

In [ ]:
cols_to_drop = ["Cabin", "Ticket", "Name"]

for df in [train_df, test_df]:
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

print(train_df.head())
print(test_df.head())


In [ ]:
train_df["Age"] = train_df["Age"].fillna(train_df["Age"].median())
test_df["Age"]  = test_df["Age"].fillna(test_df["Age"].median())

train_df["Embarked"] = train_df["Embarked"].fillna(train_df["Embarked"].mode()[0])
test_df["Fare"]       = test_df["Fare"].fillna(test_df["Fare"].median())

In [ ]:
for df in [train_df, test_df]:
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

In [ ]:
X = train_df.drop("Survived", axis=1)
y = train_df["Survived"]

In [ ]:
categorical_features = ["Sex", "Embarked", "Pclass"]
numerical_features = ["Age", "Fare", "FamilySize", "IsAlone"]

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
val_preds = model.predict(X_val)
accuracy = accuracy_score(y_val, val_preds)

print("Validation Accuracy:", accuracy)

In [ ]:
model.fit(X, y)

In [ ]:
test_preds = model.predict(test_df)

In [ ]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": test_preds
})

submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)